In [1]:
from datetime import datetime
from datetime import date
from pathlib import Path
from zoneinfo import ZoneInfo
import os
import pandas as pd
import math
import numpy as np
import scipy.io

from neuroconv.datainterfaces import TDTFiberPhotometryInterface
from neuroconv.utils import dict_deep_update, load_dict_from_file
from neuroconv.tools import configure_and_write_nwbfile
from pynwb.image import ImageSeries
from pynwb.epoch import TimeIntervals
from pynwb import TimeSeries

import tdt

from ndx_manoli_meta import AssayMetadata,IndividualMetadata

In [2]:
# load metadata table
metatable = pd.read_csv('metadata_nwb_introduction_v2.csv')
metatable.head()

# load static photometry metadata object
# currently in a cell below, but I should move it to some kind of file

,Eartag,AssayType,RecordingDate,Sex,Genotype,DOB,Family,SurgeryDate,TDTGroupPath,TDTTankPath,...,ScoreFile,PreprocessedDataPath,PreprocessedFiledFF,PreprocessedFileTrimdFF,PartnerEartag,PartnerSex,PartnerGenotype,PartnerDOB,PartnerFamily,PairDate
0,V1259,introduction,20181205,M,WT,20181001,FC,20181115,M:\photometry-Long2025\tdt_files\1 Intros,V1259-181205-134533,...,V1259 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V1259_Intro_dFF.mat,V1259_Intro_Trim_dFF.mat,NaN,NaN,NaN,NaN,NaN,20181205
1,V2210,introduction,20190218,M,WT,20181223,B5,20190125,M:\photometry-Long2025\tdt_files\1 Intros,V2210-190218-142548,...,V2210 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V2210_Intro_dFF.mat,V2210_Intro_Trim_dFF.mat,NaN,NaN,NaN,NaN,NaN,20190218
2,V2211,introduction,20190218,M,WT,20181223,B5,20190130,M:\photometry-Long2025\tdt_files\1 Intros,V2211-190218-133221,...,V2211 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V2211_Intro_dFF.mat,V2211_Intro_Trim_dFF.mat,NaN,NaN,NaN,NaN,NaN,20190218
3,V3950,introduction,20190904,M,WT,20190706,G15,20190808,M:\photometry-Long2025\tdt_files\1 Intros,V3950-190904-140232,...,V3950 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V3950_Intro_dFF.mat,V3950_Intro_Trim_dFF.mat,V4146,F,WT,20190726.0,NG5,20190904
4,V5473,introduction,20200212,M,WT,20191213,G21,20200123,M:\photometry-Long2025\tdt_files\1 Intros,V5473-200212-132254,...,V5473 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V5473_Intro_dFF.mat,V5473_Intro_Trim_dFF.mat,V5552,F,WT,20191223.0,H5,20200212


In [3]:
# LOOP OVER METADATA TABLE
# just use first line for now
i=0
folder_path = os.path.join(metatable.TDTGroupPath[i],metatable.TDTTankPath[i])
# folder_path = 'D:\\photometry_nwb\\UCSF02 NAc Photometry\\Raw Data\\1 Intros\\V2210-190218-142548'

In [4]:
folder_path

'M:\\photometry-Long2025\\tdt_files\\1 Intros\\V1259-181205-134533'

In [132]:
#### BUILD A FILE

# START TDTFiber interface
interface = TDTFiberPhotometryInterface(folder_path=folder_path, verbose=False)
metadata = interface.get_metadata()

# CHECK SESSION START TIME
sessionstart = metadata['NWBFile']['session_start_time']
# reformat
recd = str(metatable.RecordingDate[i])
sessiondate = sessionstart[0:10].replace('-','')
if sessiondate != recd:
    print(f'Session date {sessiondate} and recording date {metatable.RecordingDate[i]} mismatch for {metatable.Eartag[i]} {metatable.AssayType[i]}.')
    # break

# ADD SUBJECT METADATA

# calculate age at recording
dobstr = str(metatable.DOB[i])
dob = date(int(dobstr[0:4]),int(dobstr[4:6]),int(dobstr[6:]))
recobj = date(int(recd[0:4]),int(recd[4:6]),int(recd[6:]))
delta = recobj-dob
agei = delta.days

# calculate days post pairing
pairstr = str(metatable.PairDate[i])
paird = date(int(pairstr[0:4]),int(pairstr[4:6]),int(pairstr[6:]))
delta = recobj-paird
dpostpair = delta.days

# create NWB Subject object
metadata["Subject"] = dict(
    subject_id=f"{metatable.Eartag[i]}",
    sex=f"{metatable.Sex[i]}",
    age=f"P{agei}D",
    species="Microtus ochrogaster",
    genotype=metatable.Genotype[i],
)

# ADD METADATA ABOUT EXPERIMENT
metadata["NWBFile"].update(
    session_id=f"{metatable.AssayType[i]}",
    session_description=f"Fiber photometry recording of a prairie vole interacting in an assay of type: {metatable.AssayType[i]}.",
    experiment_description="Data associated with publication: Loss of oxytocin receptor function disrupts neural signatures of pair bonding and fidelity in the nucleus accumbens.",
    experimenter=["Long, Kimberly L. P.",],
    lab="Devanand Manoli",
    institution="University of California, San Francisco",
)

# ADD PHOTOMETRY METADATA
metadata = dict_deep_update(metadata, fiber_photometry_metadata)           

# RUN CONVERSION WITH IN-MEMORY FILE
nwbfile = TDTFiberPhotometryInterface.create_nwbfile(interface,metadata=metadata)

# ADD LAB SPECIFIC METADATA
# handle cases where the information is not recorded in the metadata
atype = metatable.AssayType[i]
if math.isnan(metatable.PartnerEartag[i]):
    pID = 'unknown'
else:
    pID = metatable.PartnerEartag[i]
pGT = 'WT'
if math.isnan(metatable.PartnerSex[i]):
    if metatable.Sex[i]=='F':
        pSex='M'
    else:
        pSex='F'
else:
    pSex = metatable.PartnerSex[i]
if math.isnan(metatable.PartnerFamily[i]):
    pFam = 'unknown'
else:
    pFam = metatable.PartnerFamily[i]
if math.isnan(metatable.PartnerDOB[i]):
    pDOB = 'unknown'
else:
    pDOB = str(metatable.PartnerDOB[i])

# create object and add to NWB file object
labmeta = make_metadata_object(atype,pID,pGT,pSex,pFam,pDOB,dpostpair)
nwbfile.add_lab_meta_data(lab_meta_data=labmeta)

# ADD VIDEO INFORMATION
# !!! write a utility to copy all of the video files from tanks to another directory for uploading
path_to_bulk_videos = os.path.join('..','behavior_videos')

# get video file path from TDT tank
vids = []
vids += [each for each in os.listdir(folder_path) if each.endswith('.avi')]
path_to_vid = os.path.join(path_to_bulk_videos,vids[0])

# get video timestamps from TDT
tstampstruct = tdt.read_block(folder_path,evtype=['epocs'])
tstamps = tstampstruct.epocs.Cam1.onset
video_ext_file = ImageSeries(
    name='behavior_video',
    description='Raw original video collected by TDT RZ5P during photometry recording and timestamped synchronously to fluorescence recording.',
    unit='n.a.',
    external_file=[path_to_vid],
    format='external',
    timestamps=tstamps,
)

# add to NWB file
nwbfile.add_acquisition(video_ext_file)

# ADD SCORING TABLE
fullscore = os.path.join(metatable.ScorePath[i],metatable.ScoreFile[i]) # get path
anno = pd.read_csv(fullscore,header=15,usecols=['Time','Behavior','Status']) # load BORIS output csv
organno = organize_BORIS_events(anno) # reformat annotation table to match required structure of TimeIntervals

# make the NWB TimeIntervals object
BORIS_annotations = TimeIntervals(
    name="raw_manual_behavior_annotations",
).from_dataframe(organno,"raw_manual_behavior_annotations",table_description="Full description of animal behavior during video, manually annotated in BORIS software. POINT events are handled as durationless and start and stop at the same time.")

nwbfile.add_time_intervals(BORIS_annotations) # add to file

# ADD PREPROCESSED DATA

# add a processing module to the file to contain intermediate processing steps
mod = nwbfile.create_processing_module(
    "Preprocessing", "Processing of fiber photometry data, including demodulation and normalization to control signal."
)

# put the dFF into a time series and add to file
ts = TimeSeries(name="dFF", data=mat['dff'][0,:], unit="au", timestamps=mat['tstamps'][0,:],description='Full dF over F calculation for the whole assay.')
mod.add_container(ts)

# WRITE FULLY FILLED OUT FILE
configure_and_write_nwbfile(
    nwbfile, nwbfile_path=f"test_write_{metatable.Eartag[i]}_{metatable.AssayType[i]}.nwb", backend="hdf5"
)

read from t=0s to t=3543.61s


C:\Users\Nerissa\anaconda3\envs\neuroconv\Lib\site-packages\hdmf\build\objectmapper.py:267: DtypeConversionWarning: Spec 'TimeSeries/timestamps': Value with data type float32 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
C:\Users\Nerissa\anaconda3\envs\neuroconv\Lib\site-packages\hdmf\build\objectmapper.py:267: DtypeConversionWarning: Spec 'TimeSeries/timestamps': Value with data type float32 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
C:\Users\Nerissa\anaconda3\envs\neuroconv\Lib\site-packages\hdmf\build\objectmapper.py:267: DtypeConversionWarning: Spec 'TimeSeries/timestamps': Value with data type float32 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


In [109]:
fullmat = os.path.join(metatable.PreprocessedDataPath[i],metatable.PreprocessedFiledFF[i])
mat = scipy.io.loadmat(fullmat)
usage=mat['tstamps'][0,0:10]

In [128]:
# PROCESSING MODULE USAGE FROM NWB

# mod = nwbfile.create_processing_module(
#     "Preprocessing", "Processing of fiber photometry data, including demodulation and normalization to control signal."
# )

ts2 = TimeSeries(name="dFF", data=mat['dff'][0,:], unit="au", timestamps=mat['tstamps'][0,:],description='Full dF over F calculation for the whole assay.')

mod.add_container(ts2)

ValueError: Cannot add <class 'pynwb.base.TimeSeries'> 'dFF' at 0x2265668767680 to dict attribute 'data_interfaces' in <class 'pynwb.base.ProcessingModule'> 'Preprocessing'. <class 'pynwb.base.TimeSeries'> 'dFF' at 0x2265676345936 already exists in 'data_interfaces' and has the same name.

In [130]:
nwbfile

root pynwb.file.NWBFile at 0x2265676392784
Fields:
  acquisition: {
    all_TDT_data <class 'abc.FiberPhotometryResponseSeries'>,
    behavior_video <class 'pynwb.image.ImageSeries'>
  }
  devices: {
    NAcc_GCaMP_injection <class 'abc.Indicator'>,
    NAcc_tdTomato_injection <class 'abc.Indicator'>,
    dichroic_mirror <class 'abc.DichroicMirror'>,
    excitation_source_calcium_signal <class 'abc.ExcitationSource'>,
    excitation_source_red_control_signal <class 'abc.ExcitationSource'>,
    optical_fiber <class 'abc.OpticalFiber'>,
    photodetector_green <class 'abc.Photodetector'>,
    photodetector_red <class 'abc.Photodetector'>
  }
  experiment_description: Data associated with publication: Loss of oxytocin receptor function disrupts neural signatures of pair bonding and fidelity in the nucleus accumbens.
  experimenter: ['Long, Kimberly L. P.']
  file_create_date: [datetime.datetime(2025, 10, 23, 14, 32, 42, 216248, tzinfo=tzlocal())]
  identifier: 2eb192d7-c856-4a52-b826-f0aa1f758b54
  institution: University of California, San Francisco
  intervals: {
    raw_manual_behavior_annotations <class 'pynwb.epoch.TimeIntervals'>
  }
  lab: Devanand Manoli
  lab_meta_data: {
    fiber_photometry <class 'abc.FiberPhotometry'>,
    vole_metadata <class 'abc.AssayMetadata'>
  }
  processing: {
    Preprocessing <class 'pynwb.base.ProcessingModule'>
  }
  session_description: Fiber photometry recording of a prairie vole interacting in an assay of type: introduction.
  session_id: introduction
  session_start_time: 2018-12-05 21:45:35.999999+00:00
  source_script: Created using NeuroConv v0.8.1
  source_script_file_name: C:\Users\Nerissa\anaconda3\envs\neuroconv\Lib\site-packages\neuroconv\basedatainterface.py
  subject: subject pynwb.file.Subject at 0x2265662979152
Fields:
  age: P65D
  age__reference: birth
  genotype: WT
  sex: M
  species: Microtus ochrogaster
  subject_id: V1259

  timestamps_reference_time: 2018-12-05 21:45:35.999999+00:00

In [115]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Mon Sep 13 17:37:46 2021',
 '__version__': '1.0',
 '__globals__': [],
 'tstamps': array([[2.0971520e-02, 3.1457279e-02, 4.1943040e-02, ..., 3.5435681e+03,
         3.5435789e+03, 3.5435891e+03]], shape=(1, 337942), dtype=float32),
 'signal': array([[0.18284953, 0.24458548, 0.3760333 , ..., 0.        , 0.        ,
         0.        ]], shape=(1, 337942), dtype=float32),
 'dF': array([[-81.8579965 , -81.56828844, -81.27622015, ...,  20.22344193,
          19.88269973,  19.52912779]], shape=(1, 337942)),
 'dff': array([[-0.99688093, -0.99685095, -0.99682095, ...,  0.23883225,
          0.23407024,  0.22922422]], shape=(1, 337942)),
 'control': array([[0.04042841, 0.08492656, 0.10585743, ..., 0.        , 0.        ,
         0.        ]], shape=(1, 337942), dtype=float32),
 'frameTimes': array([[1.01793790e+00],
        [1.06512380e+00],
        [1.11493111e+00],
        ...,
        [3.54341863e+03],
        [3.54346664e

In [13]:
# STATIC FIBER PHOTOMETRY METADATA

fiber_photometry_metadata = {
    "Ophys": {
        "FiberPhotometry": {
            "OpticalFibers": [
                {
                    "name": "optical_fiber",
                    "model": "MFC_400-430-0.48_6.5mm_MF1.25_FLT",
                    "numerical_aperture": 0.48,
                    "core_diameter_in_um": 400.0
                }
            ],
            "ExcitationSources": [
                {
                    "name": "excitation_source_calcium_signal",
                    "model": "Doric_Lenses_Connectorized_Single_LED_Blue_465",
                    "illumination_type": 'LED',
                    "excitation_wavelength_in_nm": 465.0
                },
                {
                    "name": "excitation_source_red_control_signal",
                    "model": "Doric_Lenses_Connectorized_Single_LED_Lime_560",
                    "illumination_type": 'LED',
                    "excitation_wavelength_in_nm": 560.0
                }
            ],
            "Photodetectors": [
                {
                    "name": "photodetector_green",
                    "model": "NPM_2151",
                    "detector_type": 'photodiode',
                    "detected_wavelength_in_nm": 520.0
                },
                {
                    "name": "photodetector_red",
                    "model": "NPM_2151",
                    "detector_type": 'photodiode',
                    "detected_wavelength_in_nm": 630.0
                }
            ],
            "DichroicMirrors": [
                {
                    "name": "dichroic_mirror",
                    "model": "FMC5_E1(460-490)_F1(500-540)_E2(550-580)_F2(600-680)_S"
                }
            ],
            "BandOpticalFilter":[
                {
                    "name":  "5 port Fluorescence Mini Cube (F1)",
                    "center_wavelength_in_nm": 520.0,
                    "bandwidth_in_nm": 40.0,
                    "filter_type": "bandpass",
                    "model": "FMC5_E1(460-490)_F1(500-540)_E2(550-580)_F2(600-680)_S"
                },
                {
                    "name":  "5 port Fluorescence Mini Cube (F2)",
                    "center_wavelength_in_nm": 640.0,
                    "bandwidth_in_nm": 80.0,
                    "filter_type": "bandpass",
                    "model": "FMC5_E1(460-490)_F1(500-540)_E2(550-580)_F2(600-680)_S"
                },  
            ],
            "Indicators": [
                {
                    "name": "NAcc_GCaMP_injection",
                    "description": "AAV8-Syn-GCaMP6m-WPRE viral vector used to record Ca2+ fluctuations (1.00e+13, Vigene).",
                    "label": "GCaMP6m",
                    "injection_location": "NAcc shell",
                    "injection_coordinates_in_mm": [1.7,1.2,-5.05],
                },
                {
                    "name": "NAcc_tdTomato_injection",
                    "description": "AAV8-CAG-tdTomato viral vector used as a static control (5.00e+12, Dr. Ed Boyden lab via UNC Vector Core).",
                    "label": "tdTomato",
                    "injection_location": "NAcc shell",
                    "injection_coordinates_in_mm": [1.7,1.2,-5.05],
                }
            ],
            "FiberPhotometryTable": {
                "name": "fiber_photometry_table",
                "description": "Fiber photometry system metadata table.",
                "rows": [
                    {
                        "name": "0",
                        "location": "NAcc",
                        "excitation_wavelength_in_nm": 465.0,
                        "emission_wavelength_in_nm": 525.0,
                        "indicator": "NAcc_GCaMP_injection",
                        "optical_fiber": "optical_fiber",
                        "excitation_source": "excitation_source_calcium_signal",
                        "photodetector": "photodetector_green",
                        "dichroic_mirror": "dichroic_mirror"
                    }
                ]
            },
            "FiberPhotometryResponseSeries": [
                {
                    "name": "all_TDT_data",
                    "description": "Photometry recordings were conducted with TDT RZ5P acquisition system and Synapse software. Data is 4xn with row 0 corresponding to voltage data from GCaMP excitation LED, row 1 -> tdTomato LED, row 2 -> GCaMP response data, row 3 -> tdTomato response data.",
                    "stream_name": "Fi1r",
                    "unit": "a.u.",
                    "fiber_photometry_table_region": [0],
                    "fiber_photometry_table_region_description": "The region of the FiberPhotometryTable corresponding to the calcium signal."
                }
            ]
        }
    }
}

In [14]:
def make_metadata_object(atype,pID,pGT,pSex,pFam,pDOB,dpostpair,sID=None,sSex=None,sFam=None,sDOB=None,chamber=None):
        
    match atype:
        case 'introduction':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Implanted animal is introduced to a novel partner in a clean cage.',
                assay_type__days_post_pairing=dpostpair,
            )
            
        case 'timed_mating':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Implanted animal and cohabitating partner are allowed free access after ~24 hours separated by a barrier.',
                assay_type__days_post_pairing=dpostpair,
            )
                      
           
        case 'PPT':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=10800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Implanted animal is allowed to freely move through a 3-chamber apparatus with cohabitating partner tethered in one chamber and a novel opposite-sex stranger tethered in opposite chamber.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__stranger_ID=sID,
                assay_type__stranger_GT='WT',
                assay_type__stranger_sex=sSex,
                assay_type__stranger_fam=sFam,
                assay_type__stranger_DOB=sDOB,
                assay_type__PPT_lane=1,
                assay_type__partner_chamber=chamber,
            )

        case 'separation_reun':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Focal animal remains in home cage while partner is removed to a clean cage for 60 mins. Partner is then returned to the cage with the focal animal.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__isolation_length=3600.0,
            )

        case 'aggression':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1200.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Focal animal remains in home cage while partner is removed to a clean cage for 60 mins. A novel opposite-sex stranger is then introduced to the cage with the focal animal.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__stranger_ID=sID,
                assay_type__stranger_GT='WT',
                assay_type__stranger_sex=sSex,
                assay_type__stranger_fam=sFam,
                assay_type__stranger_DOB=sDOB,
                assay_type__isolation_length=3600.0,
            )
            
    return metaObj

In [95]:
# UTILITY TO ARRANGE THE BEHAVIOR ANNOTATIONS WITH A START AND STOP FOR A SINGLE ENTRY

def organize_BORIS_events(anno):
    '''Pass a pandas dataframe of BORIS output with START and STOPs for state events in different entries.  Rearrange into a new table that has
    begin_time and stop_time specified in one line for each event.'''
    
    # filter point events
    points=anno[anno.Status=='POINT'].reset_index(drop=True)
    # filter to starts
    starts=anno[anno.Status=='START'].reset_index(drop=True)
    # filter to stops
    stops=anno[anno.Status=='STOP'].reset_index(drop=True)
    
    # if starts and stops match, just join on the event description
    if np.sum(starts.Behavior==stops.Behavior)==(anno.shape[0]-points.shape[0])/2:
        starts.rename(columns={'Time':'start_time'}, inplace=True) # rename column to match NWB specs
        stops.rename(columns={'Time':'stop_time'}, inplace=True)
        points.rename(columns={'Time':'start_time'}, inplace=True)
        points = points.assign(stop_time=points.start_time) # set POINT events to stop at the same time they start
        points= points[['start_time','stop_time','Behavior']] # reorder columns
        merged = starts.assign(stop_time=stops.stop_time) # add end times to start times
        merged = merged[['start_time','stop_time','Behavior']]
        allmerged = pd.concat([merged,points]).sort_values('start_time').reset_index(drop=True) # add point events and sort by time
    else:
        print('START and STOP events do not match. Cannot automatically reprocess table.')
        allmerged = []

    return allmerged

In [49]:
points=anno[anno.Status=='POINT'].reset_index()
states=anno[anno.Status!='POINT'].reset_index()
starts=anno[anno.Status=='START'].reset_index()
stops=anno[anno.Status=='STOP'].reset_index()

In [48]:
np.sum(starts.Behavior==stops.Behavior)

np.int64(306)

In [51]:
np.sum(starts.Behavior==stops.Behavior)==(anno.shape[0]-points.shape[0])/2

np.True_

In [46]:
print(anno.shape[0])
print(points.shape[0])

658
46


In [89]:
starts.rename(columns={'Time':'begin_time'}, inplace=True)
stops.rename(columns={'Time':'stop_time'}, inplace=True)
points.rename(columns={'Time':'begin_time'}, inplace=True)
points = points.assign(stop_time=points.begin_time)
points= points[['begin_time','stop_time','Behavior']]
merged = starts.assign(stop_time=stops.stop_time)
merged = merged[['begin_time','stop_time','Behavior']]
allmerged = pd.concat([merged,points]).sort_values('begin_time').reset_index(drop=True)

In [90]:
allmerged.head(15)

,begin_time,stop_time,Behavior
0,0.00,539.399,Don't Score
1,539.40,539.400,Start assay
2,539.40,539.549,No interaction
3,539.55,539.699,Investigate
4,539.70,559.249,No interaction
5,559.25,559.399,Sniff
6,559.40,560.699,No interaction
7,560.70,560.849,Sniff
8,560.85,563.299,No interaction
9,563.30,563.399,Investigate


In [82]:
merged.head(15)

,begin_time,stop_time,Behavior
0,0.00,539.399,Don't Score
1,539.40,539.549,No interaction
2,539.55,539.699,Investigate
3,539.70,559.249,No interaction
4,559.25,559.399,Sniff
5,559.40,560.699,No interaction
6,560.70,560.849,Sniff
7,560.85,563.299,No interaction
8,563.30,563.399,Investigate
9,563.40,563.949,Huddle


In [79]:
anno.head(20)

,Time,Behavior,Status
0,0.000,Don't Score,START
1,539.399,Don't Score,STOP
2,539.400,No interaction,START
3,539.400,Start assay,POINT
4,539.549,No interaction,STOP
5,539.550,Investigate,START
6,539.699,Investigate,STOP
7,539.700,No interaction,START
8,559.249,No interaction,STOP
9,559.250,Sniff,START
